In [6]:
# !pip install kneed
# !pip install scipy
# !pip install pandas
# !pip install matplotlib

In [9]:
import pandas as pd
from scipy.stats import ttest_ind, ttest_rel
# from kneed import KneeLocator

In [ ]:
# Ensure to update the constances: trials and episodes_per_trial variables
def find_convergence(df):
    """
    Just plug in any algorithm for convergence calculation here.
    """
    kn = KneeLocator(df['Episode'], df["AvgRewards"], curve='concave', direction='increasing')
    return kn.knee

def convergence_episodes(q):
    lower_value = 0
    upper_value = episodes_per_trial
    trial_list = []
    convergence_point = []
    for count, i in enumerate(range(trials)):
        current_df = q.iloc[lower_value: upper_value]
        lower_value += episodes_per_trial
        upper_value += episodes_per_trial
        trial_list.append(count + 1)
        convergence_point.append(find_convergence(current_df))
    return trial_list, convergence_point

In [131]:
def convergenceEstimator(sarsa, q, episodes, tolerance):
    sarsa_df = pd.read_csv(sarsa, index_col = 0)
    q_df = pd.read_csv(q, index_col = 0)
    totalRows = int(len(sarsa_df))
    sarsaConvergedEps = []
    QConvergedEps = []
    sarsa_idealRewards = sum(sarsa_df['AvgRewards'][totalRows-100:totalRows]) / 100
    q_idealRewards = sum(q_df['AvgRewards'][totalRows-100:totalRows]) / 100
    window = 12
    # get the convergence episode for sarsda
    run = 1
    epi = 0
    while epi < len(sarsa_df['Episode']):
        if (sarsa_df['AvgRewards'][epi] ) > sarsa_idealRewards-tolerance and \
            (sarsa_df['AvgRewards'][epi]) < sarsa_idealRewards+tolerance:
            sarsaConvergedEps.append(epi+1 - ((run-1) * episodes))
            epi = run * episodes
            run += 1
            continue
        epi += 1

    # get the convergence episode for sarsda
    run = 1
    epi = 0
    while epi < len(q_df['Episode']):
        if (q_df['AvgRewards'][epi] ) > q_idealRewards-tolerance and \
            (q_df['AvgRewards'][epi] ) < q_idealRewards+tolerance:
            QConvergedEps.append(epi+1 - ((run-1) * episodes))
            epi = run * episodes
            run += 1
            continue
        epi += 1
    sarsaConvergedEps, QConvergedEps = sorted(sarsaConvergedEps)[:5], sorted(QConvergedEps)[::-1][:5]
    return sarsaConvergedEps, QConvergedEps

In [163]:
import pandas as pd

sarsaCSV = 'csvs/TrueOnlineSarsaAgent_genLarge4G1_SimpleExtractor_0-6.csv'
QCSV = 'csvs/ApproximateQAgent_genLarge4G1_SimpleExtractor_0-6.csv'
sarsaConvergedEps, QConvergedEps = convergenceEstimator(sarsaCSV, QCSV, 2000, 50)


In [167]:
res = ttest_rel(sarsaConvergedEps, QConvergedEps)
df = pd.DataFrame({'Trials': [_ for _ in range(len(sarsaConvergedEps))], 'SARSAConvergence': sarsaConvergedEps, 'QConvergence': QConvergedEps, 'pVal': res.pvalue})
df.to_csv('genLarge4G1statsRes.csv')
print(res.pvalue)

0.04517647517590429


In [97]:
import glob

In [187]:
def runAvgedTtest(layout_case):
    csvFolder = f"csvs/"
    files = list(glob.glob(csvFolder+'/*.csv'))
    trueOnlineSarsaFiles = []
    ApproximateQAgentFiles = []
    allSarsaConv = []
    allQConv = []
    for file in files:
        if layout_case in file:
            if 'TrueOnlineSarsaAgent' in file:
                trueOnlineSarsaFiles.append(file)
            if 'ApproximateQAgent' in file:
                ApproximateQAgentFiles.append(file)
    for file in trueOnlineSarsaFiles:
        approxQAgent = file.replace('TrueOnlineSarsaAgent', 'ApproximateQAgent')
        if approxQAgent in ApproximateQAgentFiles:
            sarsaConvergedEps, QConvergedEps = convergenceEstimator(file, approxQAgent, 2000, 50)
            allSarsaConv.extend(sarsaConvergedEps)
            allQConv.extend(QConvergedEps)
    return allSarsaConv, allQConv

In [200]:
sarsaConvergedEps, QConvergedEps = runAvgedTtest('genMedium')

In [201]:
res = ttest_rel(sarsaConvergedEps, QConvergedEps)
df = pd.DataFrame({'Trials': [_ for _ in range(len(sarsaConvergedEps))], 'SARSAConvergence': sarsaConvergedEps, 'QConvergence': QConvergedEps, 'pVal': res.pvalue})
df.to_csv('genMedium.csv')
print(res.pvalue)

0.001933503410088491
